In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
import string
import random
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import pandas as pd
import numpy as np

In [ ]:
def parse(path):
    """
    Reads the file line by line and uses eval() to parse
    Python-dictionary-style lines (single quotes, u'' strings).
    """
    g = gzip.open(path, 'rb')
    for l in g:
        # yield eval(l) handles both {'a':1} and {u'a':1}
        yield eval(l)

# print("Loading Bundle Data...")
# bundles = []
# for line in parse('/content/bundle_data.json.gz'):
#     bundles.append(line)
# df_bundles = pd.DataFrame(bundles)
# print(f"Successfully Loaded {len(df_bundles)} bundles.")


print("Loading Game Metadata...")
games_metadata = []
for line in parse('/content/steam_games.json.gz'):
    if 'id' in line:
        games_metadata.append(line)
df_meta = pd.DataFrame(games_metadata)
print(f"Successfully Loaded {len(df_meta)} game metadata.")


# print("Loading User Reviews...")
# user_reviews = []
# count = 0
# for line in parse('/content/australian_user_reviews.json.gz'):
#     user_reviews.append(line)
#     count += 1
#     if count > 150000: break

# df_reviews = pd.DataFrame(user_reviews)
# # Flatten reviews (One row per review, not per user)
# review_data = []
# for idx, row in df_reviews.iterrows():
#     u_id = row['user_id']
#     for r in row['reviews']:
#         review_data.append({
#             'user_id': u_id,
#             'item_id': r['item_id'],
#             'recommend': r['recommend'],
#             'review_text': r['review']
#         })
# df_flat_reviews = pd.DataFrame(review_data)
# print(f"Loaded {len(df_flat_reviews)} individual reviews.")


print("Loading User Items (Ownership & Playtime)...")
data_items = []
limit = 150000  #Increase this if you have more RAM.
count = 0

for line in parse('/content/australian_users_items.json.gz'):
    data_items.append(line)
    count += 1
    if count > limit:
        break
df_items = pd.DataFrame(data_items)
print(f"Successfully Loaded {len(df_items)} users.")